In [206]:
import pandas as pd
import janitor

In [207]:
# Group and keep columns with 'as_index'
# https://stackoverflow.com/questions/31569549/how-to-groupby-a-dataframe-in-pandas-and-keep-columns

In [208]:
data = pd.read_csv('filter.csv')

In [209]:
print(data.set_index(['ID_REGISTRO']).index.is_unique) # ids are not unique... it's unclear how reports are aggregated

False


In [210]:
data.keys()

Index(['ID_REGISTRO', 'ENTIDAD_UM', 'ENTIDAD_RES', 'FECHA_INGRESO',
       'RESULTADO'],
      dtype='object')

In [211]:
data['FECHA_INGRESO'] = pd.to_datetime(data.FECHA_INGRESO, format='%Y-%m-%d', errors='coerce')
data['FECHA_INGRESO'] = data['FECHA_INGRESO'].dt.strftime('%Y-%m-%d')
data = data.set_index(pd.DatetimeIndex(data['FECHA_INGRESO'])).sort_index()

### Note
##### Daily report for april 20 (ie, reported in 21 april) stops using testing site
##### So we shall treat them accordingly

In [212]:
tbl1 = data.filter_date('FECHA_INGRESO','2020-01-06','2020-04-20')
tbl1 = tbl1.drop(['ENTIDAD_RES'], axis=1)
tbl1 = tbl1.set_index(['FECHA_INGRESO', 'ENTIDAD_UM', 'ID_REGISTRO']) # Here we use region of testing

In [213]:
# Health testing site (positive) cases by date
tbl1 = tbl1.groupby(['FECHA_INGRESO','ENTIDAD_UM','ID_REGISTRO'], as_index=True).count().reset_index()

In [214]:
tbl2 = data.filter_date('FECHA_INGRESO','2020-04-21','2020-04-25')
tbl2 = tbl2.drop(['ENTIDAD_UM'], axis=1)
tbl2 = tbl2.set_index(['FECHA_INGRESO', 'ENTIDAD_RES', 'ID_REGISTRO']) # Here we use region of residence

In [215]:
# Residence cases (positive) by date
tbl2 = tbl2.groupby(['FECHA_INGRESO','ENTIDAD_RES','ID_REGISTRO'], as_index=True).count().reset_index()

In [216]:
# Join tables

In [217]:
tbl1 = tbl1.rename(columns={'ENTIDAD_UM':'Region_ID'})

In [218]:
tbl2 = tbl2.rename(columns={'ENTIDAD_RES':'Region_ID'})

In [219]:
tbl_join = pd.concat([tbl1, tbl2])

In [220]:
# Friendly colnames

In [221]:
data.keys()

Index(['ID_REGISTRO', 'ENTIDAD_UM', 'ENTIDAD_RES', 'FECHA_INGRESO',
       'RESULTADO'],
      dtype='object')

In [222]:
tbl_join = tbl_join.rename(columns={'FECHA_INGRESO':'Date_Confirmed', 'ID_REGISTRO':'Case_ID', 'RESULTADO':'Status'})

In [223]:
len(tbl_join) # Total cases would have been less if we didn't take into account the change in test reporting :o

14039

In [224]:
tbl_join['Status'] = '1'

In [225]:
tbl_join.to_csv('confirmed_by_date.csv', index=False)